In [51]:
import itertools
import numpy as np
import math

Diagonal

In [203]:
# input: migration rate matrix
# for two-pop, use anti-diagonal
migmat = np.array([["AA", "AB"], ["BA", "BB"]])
np.diagonal(np.fliplr(migmat))

array(['AB', 'BA'], dtype='<U2')

In [46]:
def q_diagonal(popsizes, migration_rates):
    assert len(popsizes) == len(migration_rates)

    mig_rate_combinations = list(itertools.combinations(migration_rates, 2))

    diagonal =  ([-(1/popsizes[i] + migration_rates[i]) for i in range(len(popsizes))] 
                 + [-(sum(mig_rate_combinations[i])/2) for i in range(len(mig_rate_combinations))] 
                 + [0])

    return np.diag(diagonal)


In [50]:
q = q_diagonal([1,1], [0.5,0])
q

array([[-1.5 ,  0.  ,  0.  ,  0.  ],
       [ 0.  , -1.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  , -0.25,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ]])

Migration rate matrix

In [272]:
migmat = np.array(
    [
        ["AA", "AB", "AC", "AD", "AE"],
        ["BA", "BB", "BC", "BD", "BE"],
        ["CA", "CB", "CC", "CD", "CE"],
        ["DA", "DB", "DC", "DD", "DE"],
        ["EA", "EB", "EC", "ED", "EE"]
    ]
)

migmat

array([['AA', 'AB', 'AC', 'AD', 'AE'],
       ['BA', 'BB', 'BC', 'BD', 'BE'],
       ['CA', 'CB', 'CC', 'CD', 'CE'],
       ['DA', 'DB', 'DC', 'DD', 'DE'],
       ['EA', 'EB', 'EC', 'ED', 'EE']], dtype='<U2')

In [273]:
# two pops
qm2 = np.array([
    [migmat[0,1],
     migmat[1,0]]
])

qm2

array([['AB', 'BA']], dtype='<U2')

In [274]:
# three pops
qm3 = np.array([
    [migmat[0,1], migmat[0,2], 0],
    [migmat[1,0], 0, migmat[1,2]],
    [0, migmat[2,0], migmat[2,1]]
])

qm3

array([['AB', 'AC', '0'],
       ['BA', '0', 'BC'],
       ['0', 'CA', 'CB']], dtype='<U21')

In [275]:
d_matrix_diag = [migmat[0,3], migmat[1,3], migmat[2,3]]
d_matrix_bottom = [migmat[3,0], migmat[3,1], migmat[3,2]]

d = np.diag(d_matrix_diag)
d = np.r_[d, [d_matrix_bottom]]

In [285]:
def deme_submatrix(migmat, deme_index):
    qm = np.diag([migmat[i, deme_index] for i in range(deme_index)])
    bottom = [migmat[deme_index, i] for i in range(deme_index)]

    qm = np.r_[qm, [bottom]]
    return qm



In [334]:
def qmig_submatrices(migmat, n_pops):

    submatrices = []

    for deme_idx in range(n_pops-1):
        m = deme_submatrix(migmat, deme_idx+1)
        submatrices.append(m)

    return submatrices

In [335]:
def pad_submatrices(submatrices):
    max_shape = submatrices[-1].shape

    padded = [np.pad(submat, ((0, max_shape[0]-submat.shape[0]),
                        (0, 0)),
                        'constant', constant_values=0) for submat in submatrices]
    
    return padded

In [336]:
def qmig(migmat, n_pops):
    """Generate submatrix that describes migration rates in terms of a generator matrix"""
    submatrices = qmig_submatrices(migmat, n_pops)
    padded_submatrices = pad_submatrices(submatrices)
    return np.concatenate(pad_submatrices(submatrices), axis=1)
    

In [340]:
qmig(migmat, 4)

array([['AB', 'AC', '', 'AD', '', ''],
       ['BA', '', 'BC', '', 'BD', ''],
       ['0', 'CA', 'CB', '', '', 'CD'],
       ['0', '0', '0', 'DA', 'DB', 'DC']], dtype='<U2')

Complementary migration matrix

In [347]:
qmig(migmat, 3)

array([['AB', 'AC', ''],
       ['BA', '', 'BC'],
       ['0', 'CA', 'CB']], dtype='<U2')